# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from datetime import datetime

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("/home/workspace/data/song_data/")

In [5]:
filepath = song_files[10]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARKRRTF1187B9984DA,NaN,,NaN,Sonora Santanera,177.47546,1,SOXVLOJ12AB0189215,Amor De Cabaret,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]
song_data 

array(['SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0,
       177.47546], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude' ]].values[0]
artist_data

array(['ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')


In [12]:
filepath = log_files[3]


In [13]:
df = pd.read_json(filepath, lines=True)


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df.page == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Future Sound of London,Logged In,Ryan,M,0,Smith,405.28934,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Papua New Guinea,200,1543020300796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,Harmonia,Logged In,Ryan,M,1,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Sehr kosmisch,200,1543020705796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
9,My Chemical Romance,Logged In,Dustin,M,7,Lee,311.11791,free,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",PUT,NextSong,1.540853e+12,86,Welcome To The Black Parade (Album Version),200,1543026602796,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,87
10,Disturbing Tha Peace / Ludacris / Mystikal / I-20,Logged In,Mohammad,M,0,Rodriguez,276.42730,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Move Bitch,200,1543027714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
11,Carlos PaiÃÂ£o,Logged In,Mohammad,M,1,Rodriguez,185.99138,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Playback,200,1543027990796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [15]:
t = pd.to_datetime(df['ts'],unit='ms')

df['ts'] = pd.to_datetime(df['ts'],unit='ms')

df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Future Sound of London,Logged In,Ryan,M,0,Smith,405.28934,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Papua New Guinea,200,2018-11-24 00:45:00.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,Harmonia,Logged In,Ryan,M,1,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Sehr kosmisch,200,2018-11-24 00:51:45.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
9,My Chemical Romance,Logged In,Dustin,M,7,Lee,311.11791,free,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",PUT,NextSong,1.540853e+12,86,Welcome To The Black Parade (Album Version),200,2018-11-24 02:30:02.796,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,87
10,Disturbing Tha Peace / Ludacris / Mystikal / I-20,Logged In,Mohammad,M,0,Rodriguez,276.42730,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Move Bitch,200,2018-11-24 02:48:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
11,Carlos PaiÃÂ£o,Logged In,Mohammad,M,1,Rodriguez,185.99138,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Playback,200,2018-11-24 02:53:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [16]:
timestamp = t.values
year = t.dt.year.values
hour = t.dt.hour.values
day = t.dt.day.values
week_of_year = t.dt.week.values
month = t.dt.month.values
weekday = t.dt.weekday.values


In [17]:
time_data = [timestamp, hour, day, week_of_year, month, year, weekday]


column_labels = ('timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday')

timedict = {}

for i in range(0, len(column_labels)):
    timedict[column_labels[i]] = time_data[i]

    


In [18]:
time_df = pd.DataFrame.from_dict(timedict)



#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates(subset=['userId'])


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`


#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
#df['dtvalues'] = pd.to_datetime(df['ts'], unit='ms')
#df.head()

In [25]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        print(1)
        songid, artistid = results
    else:
        print(0)
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid,  artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [47]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.